## Import libraries

In [1]:
import pandas as pd
import numpy as np
from typing import Optional, List, Tuple
import random
import time
import math
import synthetic_data_generators as sdg

import time

## Import Synthetic Data

In [2]:
synth_network = pd.read_csv("../data/synth_network.csv")
synth_pool = pd.read_csv("../data/synth_pool.csv")
synth_members = pd.read_csv("../data/synth_members.csv")
synth_reqs = sdg.synth_reqs

In [3]:
print(f"Network size: {len(synth_network)}")
print(f"Pool size: {len(synth_pool)}")
print(f"Number of members: {len(synth_members)}")
synth_reqs

Network size: 100
Pool size: 100
Number of members: 1000


,specialty,county,provider_count,distance_req,min_access_pct,min_providers
0,cardiologist,wayne,1,15,90,5
1,pcp,wayne,2,15,90,10
2,ent,wayne,1,15,90,5
3,urologist,wayne,1,15,90,5
4,obgyn,wayne,1,15,90,5


In [25]:
class NetworkOptimizer:
    """
Class that can perform provider network optimization. Implementation is a steepest-ascent, Hill-climbing local search 
optimization algorithm.

>>> optimizer = NetworkOptimizer(pool, members, adequacy_reqs)
>>> optimizer.optimize(num_rounds)
    """
    def __init__(self, 
                 pool: pd.DataFrame,
                 members: pd.DataFrame,
                 adequacy_reqs: pd.DataFrame,
                 user_objective: Optional[callable] = None,
                 network: Optional[pd.DataFrame] = None
                 ) -> None:
        """
    Initialize the optimizer. The pool, members, and adequacy requirements are required. If not passed, the network will start as blank and the optimizer
    will optimize it, guided by the objective function.
    
    :param pool: pandas DataFrame storing the pool of potential providers the network can have
    :param members: pandas DataFrame storing the members (beneficiaries) that the network serves
    :param adequacy_reqs: pandas DataFrame storing the adequacy requirements for the network
    :param objective: objective function that takes in a pandas dataframe and guides the algorithm
    :param network: pandas DataFrame storing the providers already contracted for the network, if any
        """
        
        self.initial_pool = pool.copy()
        self.adequacy_reqs = adequacy_reqs.copy()
        self.members = members.copy()
        self.user_objective = user_objective
        self.initial_network = network.copy() if network is not None else pd.DataFrame(columns=self.initial_pool.columns)
        self.distance_matrix = np.empty(0)
        self.performance_history = np.empty(0)
        self.move_tracker = []
        self.time_tracker = np.empty(0)
        self.total_optimization_rounds = 0
        self.adequacy_detail = np.empty(0)

        # create access listing
        if len(self.initial_network) > 0:
            self.access_listing = self.members.rename(
                {"latitude":"member_latitude", 
                "longitude":"member_longitude",
                "county":"member_county"}, axis=1).merge(pd.concat([self.initial_network, self.initial_pool]).rename(
                    {"latitude":"provider_latitude",
                    "longitude":"provider_longitude",
                    "county":"provider_county"}, axis=1), how="cross")

        else:
            self.access_listing = self.members.rename(
                {"latitude":"member_latitude", 
                "longitude":"member_longitude",
                "county":"member_county"}, axis=1).merge(self.initial_pool.rename(
                    {"latitude":"provider_latitude",
                    "longitude":"provider_longitude",
                    "county":"provider_county"}, axis=1), how="cross")

        def haversine(lat1, lon1, lat2, lon2):
     
            # distance between latitudes
            # and longitudes
            dLat = (lat2 - lat1) * math.pi / 180.0
            dLon = (lon2 - lon1) * math.pi / 180.0
        
            # convert to radians
            lat1 = (lat1) * math.pi / 180.0
            lat2 = (lat2) * math.pi / 180.0
        
            # apply formulae
            a = (pow(math.sin(dLat / 2), 2) +
                pow(math.sin(dLon / 2), 2) *
                    math.cos(lat1) * math.cos(lat2))
            rad = 3959
            c = 2 * math.asin(math.sqrt(a))
            return rad * c
        
        self.access_listing["distance"] = self.access_listing.apply(lambda row: haversine(row.member_latitude, 
                                                                                          row.member_longitude, 
                                                                                          row.provider_latitude, 
                                                                                          row.provider_longitude), axis=1).round(1)

        self.access_listing = self.access_listing.merge(self.adequacy_reqs, left_on=["member_county", "specialty"], right_on=["county", "specialty"], how="left")
        
        self.pct_serving = (self.access_listing[["npi"]].merge(
            (self.access_listing.loc[self.access_listing.distance <= self.access_listing.distance_req].groupby("npi")["member_id"].count() / len(self.members))
            .reset_index()
            .rename({"member_id":"pct_serving"}, axis=1), on="npi", how="left").fillna(0).drop_duplicates().reset_index(drop=True))

        self.pool = pool.copy().merge(self.pct_serving, on=["npi"], how="left")

        self.best_network = network.copy().merge(self.pct_serving, on=["npi"], how="left") if network is not None else pd.DataFrame(columns=self.pool.columns)

    def adequacy(self, network: pd.DataFrame) -> float:
        """
    Calculate adequacy of a network using the adequacy requirements provided by the user. The returned value is a float that
    is a slight modification of the adequacy index score. It takes the network and the adequacy requirements and it returns the mean
    of the product of the percent of members with access and the percent of required providers for all the county/specialty combinations.

    :param network: pandas DataFrame with the network for which you want to calculate adequacy 
        """
        if len(network) == 0:
            return 0
        else:
            network_distances = (self.access_listing[["member_id", "npi", "location_id", "distance", "provider_count"]]
                     .merge(network[["npi", "location_id", "specialty", "county"]], on=["npi", "location_id"], how="inner")
                     .merge(self.adequacy_reqs[["county", "specialty", "distance_req"]], on=["county", "specialty"], how="left"))

            network_distances["meets_distance"] = network_distances.distance <= network_distances.distance_req

            access_summary = self.adequacy_reqs.merge(
                network_distances
                .loc[network_distances.meets_distance == True]
                .groupby(["county", "specialty"])["member_id"].nunique()
                .reset_index()
                .rename({"member_id":"members_with_access"}, axis=1), on=["county", "specialty"], how="left"
            )

            return network_distances
            
            # access_summary["pct_members_with_access"] = access_summary.members_with_access / len(self.members)

            # provider_counts = network_distances.groupby(["county", "specialty"])["npi"].nunique().reset_index().rename({"npi":"provider_count"}, axis=1)

            # adequacy_detail = access_summary.merge(provider_counts, on=["county", "specialty"], how="left").fillna(0)

            # adequacy_detail["pct_req_providers"] = adequacy_detail.provider_count / adequacy_detail.min_providers
            
            # self.adequacy_detail = adequacy_detail

            # mean_adequacy_score = round((adequacy_detail.pct_members_with_access.apply(lambda x: min(x,1)) * adequacy_detail.pct_req_providers.apply(lambda x: min(x,1))).mean(), 3)

            # adequacy_county_specialties = len(adequacy_detail.loc[(adequacy_detail.pct_members_with_access * 100 >= adequacy_detail.min_access_pct)
            #                                                       & (adequacy_detail.provider_count >= adequacy_detail.min_providers)])

            # if adequacy_county_specialties == len(adequacy_detail):
            #     return 1
            # else:
            #     return mean_adequacy_score
        
    def objective(self, network: pd.DataFrame) -> float:
        """
    Objective function that describes the goal of the optimization. Takes in a pandas DataFrame storing a provider network as input.
    It is the compass for the algorithm to optimize the network. The default is adequacy, but if the user passes in a function, it will use that instead.

    :param network: pandas DataFrame with the network for which you want to calculate performance
        """
        return self.adequacy(network) if self.user_objective is None else self.user_objective(self, network)

    def successor(self, network: pd.DataFrame, pool: pd.DataFrame) -> List[Tuple]:
        """
    Returns all possible moves as successor states, given the provided network and provider pool. This represents
    all possible changes to the network. If empty network, all successor states are simply the states with each pool provider
    added to the network. Important to note that each successor state only deals with one change of a provider, i.e. the smallest
    possible "step" you can take.

    :param network: pandas DataFrame with the network in its current state
    :param pool: pandas DataFrame with the pool of potential providers
        """
        # get additions
        additions = [("addition", idx) for idx in pool.sort_values(by="pct_serving", ascending=False).index.to_list()]
        if len(network) == 0:
            removals = [(None, None) for i in range(len(additions))]
            swaps = [(None, None, None) for i in range(len(additions))]
            return list(zip(additions, removals, swaps))
        else:
            # get replacement combinations within specialty
            swapDF = network.reset_index().merge(pool.reset_index(), on = ["county", "specialty"], how = "inner", suffixes= ["_network", "_pool"]).dropna()
            swapDF["pct_diff"] = swapDF.pct_serving_pool - swapDF.pct_serving_network
            swaps = [("swap",i,j) for i,j in swapDF.sort_values(by="pct_diff", ascending=False)[["index_network", "index_pool"]].values]
            # get removals
            removals =  [("removal", idx) for idx in network.sort_values(by="pct_serving", ascending=True).index.astype(int).to_list()]
            return list(zip(additions, removals, swaps))
    
    def create_state(self, network: pd.DataFrame, pool: pd.DataFrame, change: Tuple):
        """
    Create a network based on the pool and the change. This function takes the network passed in and creates a copy, then 
    makes the change that is described by the change parameter, which comes as a tuple. The first element of the tuple
    tells you what kind of change: addition, removal, or a swap. The rest of the tuple tells you the index of the row that is associated 
    with the change.

    :param network: pandas DataFrame with the network in its current state
    :param pool: pandas DataFrame with the pool of potential providers
    :param change: tuple that describes the change to be made to the network
        """
        # make a true copy of the network, not just a reference
        # see here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html
        new_network = network.copy()

        if change[0] == "addition":
            new_network.loc[len(new_network)] = pool.loc[change[1]]
            return new_network
        elif change[0] == "removal":
            new_network = new_network.drop(change[1])
            return new_network
        else:
            new_network.loc[change[1]] = pool.loc[change[2]]
            return new_network
    
    def optimize(self, num_rounds: int) -> None:
        """
    Perform steepest-ascent local search optimization for a number of rounds input by the user.
    The algorithm takes the network, determines all the possible moves from the successor function. Then
    it calculates the performance of all the successor states and stores the best one. Repeat for number of
    rounds or until goal state is met.

    :param num_rounds: max number of rounds to perform optimization
        """
        # save performance of initial network
        if len(self.performance_history) == 0:
            self.performance_history = np.append(self.performance_history, self.objective(self.initial_network))

        # for set number of rounds
        for optim_round in range(num_rounds):
            self.total_optimization_rounds += 1
            start = time.perf_counter()
            print(f"Optimization round {self.total_optimization_rounds} ...")
            # get state changes
            state_changes = self.successor(self.best_network, self.pool)
            if len(state_changes) == 0:
                print("Pool has been exhaused")
                break
            
            for options in state_changes:
                # generate new states for the options and get the scores
                new_states = [self.create_state(self.best_network, self.pool, option) for option in options if option[0] in ["addition", "removal", "swap"]]
                new_scores = np.array([self.objective(state) for state in new_states])

                # get the best performing state and score
                best_score = np.max(new_scores)
                best_state_idx = np.argmax(new_scores)

                best_move = options[best_state_idx]
                best_state = new_states[best_state_idx]

                #if the best new move is better than the latest best performance
                if best_score > self.performance_history[-1]:
                    # store best performance and move
                    self.performance_history = np.append(self.performance_history, best_score)
                    self.move_tracker.append(best_move)
                    # update the best network
                    self.best_network = best_state
                    # if move involves a pool provider
                    # drop the new provider from pool so that the algo knows not to try that one again
                    if best_move[0] == "swap":
                        self.pool.drop(best_move[2], inplace = True)
                    elif best_move[0] == "addition":
                        self.pool.drop(best_move[1], inplace = True)
                    stop = time.perf_counter()
                    self.time_tracker = np.append(self.time_tracker, stop - start)
                    break
            if len(self.move_tracker) < self.total_optimization_rounds:
                stop = time.perf_counter()
                self.time_tracker = np.append(self.time_tracker, stop - start)
                print("No more options for optimization")
                break

            stop = time.perf_counter()
            self.time_tracker = np.append(self.time_tracker, stop - start)

        print(f"Average seconds per round of optimization: {self.time_tracker.mean().round(1)}")
        print(f"Adequacy score for best network: {self.adequacy(self.best_network)}\n")

In [26]:
optimizer = NetworkOptimizer(pool=synth_pool, 
                             members=synth_members, 
                             adequacy_reqs=synth_reqs, 
                             network=synth_network
                             )

In [28]:
network_distances = optimizer.adequacy(synth_network)

In [41]:
access_detail = (network_distances
                         .loc[network_distances.meets_distance == True]
                         .groupby(["member_id", "county", "specialty", "provider_count"])[["npi"]]
                         .count()
                         .reset_index()
                         .rename({"npi":"servicing_provider_count"}, axis=1))

member_access_summary = access_detail.loc[access_detail.servicing_provider_count >= access_detail.provider_count]

member_access_summary.groupby(["county", "specialty"])[["member_id"]].nunique().reset_index()#.rename()

,county,specialty,member_id
0,wayne,cardiologist,996
1,wayne,ent,1000
2,wayne,obgyn,996
3,wayne,pcp,965
4,wayne,urologist,990


In [17]:
optimizer.initial_network

,npi,specialty,group_id,efficiency,effectiveness,location_id,county,latitude,longitude
0,25,urologist,3,5,3,0,wayne,42.160671,-83.785516
1,59,pcp,6,5,1,1,wayne,41.978302,-83.515406
2,68,ent,4,3,3,2,wayne,42.133632,-83.801916
3,40,urologist,5,4,3,3,wayne,42.362166,-83.699214
4,12,obgyn,1,1,5,4,wayne,42.120197,-83.814452
...,...,...,...,...,...,...,...,...,...
95,39,urologist,1,2,5,95,wayne,42.382563,-83.452465
96,15,obgyn,0,2,1,96,wayne,42.136383,-83.476035
97,61,obgyn,12,3,1,97,wayne,42.309586,-83.725904
98,81,obgyn,5,1,2,98,wayne,41.972517,-83.876510
